# Leitura da base de dados original

In [1]:
import pandas as pd

dados = pd.read_csv('../data/raw/train.tsv', sep='\t')

dados.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


Quantidade de registros

In [3]:
print('Quantidade de registros = ', len(dados))
print('Quantidade de registros únicos = ', dados.name.nunique())
print('Porcentagem de registros únicos = ', 100 * dados.name.nunique()/len(dados))

Quantidade de registros =  1482535
Quantidade de registros únicos =  1225273
Porcentagem de registros únicos =  82.64715504187085


Quantidade de categorias do dataset

In [5]:
print('Números de categorias:', dados.category_name.unique().size)

Números de categorias: 1288


Descrição da coluna `price`

Quantidade de produtos com preço de anúncio igual a zero.

In [7]:
dados[dados.price == 0].size

6992

Quantidad de produtos com preço de anúncio nulo.

In [8]:
dados.price.isna().sum()

0

# Separação das categorias em diversas colunas

In [9]:
colunas = dados.category_name.str.split('/', expand=True)

In [10]:
colunas

,0,1,2,3,4
0,Men,Tops,T-shirts,None,None
1,Electronics,Computers & Tablets,Components & Parts,None,None
2,Women,Tops & Blouses,Blouse,None,None
3,Home,Home Décor,Home Décor Accents,None,None
4,Women,Jewelry,Necklaces,None,None
...,...,...,...,...,...
1482530,Women,Dresses,Mid-Calf,None,None
1482531,Kids,Girls 2T-5T,Dresses,None,None
1482532,Sports & Outdoors,Exercise,Fitness accessories,None,None
1482533,Home,Home Décor,Home Décor Accents,None,None


Quantidade de valores nulos na coluna de índice 3

In [11]:
colunas[3].isnull().sum() / len(colunas)

0.997039530264041

Quantidade de valores nulos na coluna de índice 4

In [12]:
colunas[4].isnull().sum() / len(colunas)

0.9979366423052407

# Tratamento de inconsistências

Remoção de preços com valor igual a zero.

Padronização do nome dos anúncios para apenas minúsculas.

In [ ]:
dados2 = dados.query('price > 0')
dados2.loc[:, 'name'] = dados2.name.str.lower()
dados2.shape

Função para remover caracteres especiais dos nomes.

In [21]:
from nltk.tokenize import RegexpTokenizer
import numpy as np

def clean_names(name):
    tokenizer = RegexpTokenizer(r'\w+')
    
    token = tokenizer.tokenize(name)
    name = ''
    
    for n in token:
        if not n.isdigit() and not n == 'rm':
            name += n
            name += ' '

    name = name.strip()
    
    return name

Remoção de caracters especiais da coluna nome.

In [ ]:
dados2.loc[:, 'name'] = [clean_names(name) for name in dados2.name]

Contagem das marcas com valores nulos.

In [17]:
dados2.brand_name.isna().sum()/len(dados2)

0.426775085529011

Função para preencher valores nulos.

In [18]:
def preecher_nans(dado, fill=''):
    dado[dado.isna()] = fill
    return dado 

Substituição dos valores nulos em `brand_name` por "No brand"

In [ ]:
dados2.loc[:,'brand_name'] = preecher_nans(dados2['brand_name'], fill='No Brand')

Separação da coluna `category` em sub-categorias.

In [ ]:
dados2.loc[:,'category_1'] = colunas[0]
dados2.loc[:,'category_2'] = colunas[1]
dados2.loc[:,'category_3'] = colunas[2]
dados2.drop('category_name', axis=1, inplace=True)
dados2 = dados2[['name', 'category_1', 'category_2',
       'category_3', 'item_condition_id', 'brand_name', 'price',
       'shipping', 'item_description']]

Contagem de valores nulos em `category_1`, `category_2` e `category_3`

In [24]:
dados2.category_1.isna().sum()/len(dados2)

0.0042614336207810015

In [25]:
dados2.category_2.isna().sum()/len(dados2)

0.0042614336207810015

In [26]:
dados2.category_3.isna().sum()/len(dados2)

0.0042614336207810015

In [ ]:
dados2.loc[:, 'category_1'] = preecher_nans(dados2['category_1'], fill='No category')
dados2.loc[:,'category_2'] = preecher_nans(dados2['category_2'], fill='No category')
dados2.loc[:,'category_3'] = preecher_nans(dados2['category_3'], fill='No category')

Função para geração de datas e estoques.

In [27]:
import random
import numpy as np


def data(n, seed):

    datas = []
    
    random.seed(seed)
    for i in range(n):
        dia_maximo = 32
        dia_minimo = 1
        mes_maximo = 13
        mes_minimo = 1
        
        dia = int(random.random() * (dia_maximo - dia_minimo) + dia_minimo)
        mes = int(random.random() * (mes_maximo - mes_minimo) + mes_minimo)
        
        if mes == 2:
            while dia > 28:
                dia = int(random.random() * (dia_maximo - dia_minimo) + dia_minimo)
        
        if mes in [4, 6, 9, 11]:
            while dia > 30:
                dia = int(random.random() * (dia_maximo - dia_minimo) + dia_minimo)
        
        datas.append(str(dia)+'-'+str(mes)+'-2018')
        
    return datas

def estoque(n, seed):

    np.random.seed(seed)
    mu, sigma = 1, 20
    s = np.random.normal(mu, sigma, n)
    s[s < 0] = s[s < 0] * -0.5
    s = s.astype(int)
    s[s < 1] = 1
    
    return s

dados2['date']  = data(n = dados2.shape[0], seed = 10)
dados2['stock'] = estoque(n = dados2.shape[0], seed = 10)

Padronização das descrições dos itens

Nomes para apenas minúsculas.

Remoção de caracteres especiais.

Preenchimento de nulos por "No comment yet."

In [71]:
import numpy as np
descriptions = []
for name in dados2.item_description:
    if type(name) == float:
        if np.isnan(name):
            descriptions.append('no description yet')
    else:
        descriptions.append(clean_names(name.lower()))


In [73]:
dados2.drop('item_description', axis=1, inplace=True)
dados2.head()

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,date,stock
0,mlb cincinnati reds t shirt size xl,Men,Tops,T-shirts,3,No Brand,10.0,1,18-6-2018,27
1,razer blackwidow chroma keyboard,Electronics,Computers & Tablets,Components & Parts,3,Razer,52.0,0,18-3-2018,15
2,ava viv blouse,Women,Tops & Blouses,Blouse,1,Target,10.0,1,26-10-2018,14
3,leather horse statues,Home,Home Décor,Home Décor Accents,1,No Brand,35.0,1,21-2-2018,1
4,24k gold plated rose,Women,Jewelry,Necklaces,1,No Brand,44.0,0,17-4-2018,13


In [74]:
descriptions = pd.DataFrame(descriptions)
descriptions.head()

,0
0,no description yet
1,this keyboard is in great condition and works ...
2,adorable top with a hint of lace and a key hol...
3,new with tags leather horses retail for each s...
4,complete with certificate of authenticity


In [75]:
dados2['item_description'] = descriptions[0]
dados2.head()

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,date,stock,item_description
0,mlb cincinnati reds t shirt size xl,Men,Tops,T-shirts,3,No Brand,10.0,1,18-6-2018,27,no description yet
1,razer blackwidow chroma keyboard,Electronics,Computers & Tablets,Components & Parts,3,Razer,52.0,0,18-3-2018,15,this keyboard is in great condition and works ...
2,ava viv blouse,Women,Tops & Blouses,Blouse,1,Target,10.0,1,26-10-2018,14,adorable top with a hint of lace and a key hol...
3,leather horse statues,Home,Home Décor,Home Décor Accents,1,No Brand,35.0,1,21-2-2018,1,new with tags leather horses retail for each s...
4,24k gold plated rose,Women,Jewelry,Necklaces,1,No Brand,44.0,0,17-4-2018,13,complete with certificate of authenticity


Salva arquivo com dados tratatados.

In [77]:
dados2.to_csv('train_data_prep02.csv', index=False)